# Bob

* Bob wants to receive a message from Alice.
* He can communicate with Alice over an unsecure, classical channel.
* They can also communicate over a quantum channel.

Bob establishes a classical channel to Alice. Anyone can listen in:

In [ ]:
# server = "broker.emqx.io"  # fallback server.
mqtt_broker_ip = "localhost"

channel = "quantum_cryptography/classical_channel"

import paho.mqtt.client as mqtt

# The callback for when the client receives a CONNACK response from the server.
def on_connect(client, userdata, flags, rc):
    print("Connected with result code " + str(rc))

    # Subscribing in on_connect() means that if we lose the connection and
    # reconnect then subscriptions will be renewed.
    client.subscribe(channel)


# The callback for when a PUBLISH message is received from the server.
def on_message(client, userdata, msg):
    print("received:\t" + msg.topic + " " + str(msg.payload))


client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_broker_ip, 1883, 60)

**Step 1**: Bob listens for messages from Alice:

In [ ]:
client.loop_forever()